# Calculando SVF para um ponto com Pandas

Neste experimento vamos calcular para um ponto definido o SVF utilizando Pandas. Para isso será utilizado uma técnica de calcular para cada ponto o $\theta$ e o $\phi$ a partir do ponto escolhido e então agrupar pelo particionamento da semiesfera definida

In [1]:
import pandas as pd
import geopandas as gpd
import pdal
import numpy as np

ModuleNotFoundError: No module named 'geopandas'

In [223]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


In [225]:
pipeline="""{
  "pipeline": [
    {
        "type": "readers.las",
        "filename": "arquivos/MDS/MDS_3314-231.laz"
    },
    {
        "type": "filters.sample",
        "radius": 1
    }
  ]
}"""

# pipeline="""{
#   "pipeline": [
#     {
#         "type": "readers.las",
#         "filename": "arquivos/MDS/MDS_3314-231.laz"
#     }
#   ]
# }"""

In [226]:
gdf_observ = gpd.read_file('arquivos/pontos_validacao/pontos_preliminares.gpkg')

In [227]:
# gdf_observ = gdf_observ.to_crs(31983)

In [228]:
gdf_observ

,SCM,geometry
0,3314-231,POINT Z (333344.322 7395282.674 724.891)
1,3331-132,POINT Z (324531.390 7385905.216 757.406)
2,4431-222,POINT Z (352135.100 7386243.508 789.996)
3,3314-231,POINT Z (333349.447 7395315.760 724.728)
4,3314-231,POINT Z (333353.608 7395345.881 724.915)
5,3314-231,POINT Z (333367.084 7395363.517 724.698)
6,3314-231,POINT Z (333354.996 7395284.648 724.802)
7,3314-231,POINT Z (333357.175 7395273.947 724.968)
8,3314-231,POINT Z (333347.862 7395272.560 724.917)
9,3314-231,POINT Z (333361.337 7395326.659 724.808)


In [229]:
[list(x.coords)[0][2] for x in gdf_observ.geometry]

[724.8905029296875,
 757.4063110351562,
 789.9957275390625,
 724.7280883789062,
 724.9146118164062,
 724.698486328125,
 724.8023071289062,
 724.9680786132812,
 724.916748046875,
 724.8078002929688,
 724.7515258789062,
 724.8031005859375,
 724.9634399414062,
 724.8020629882812,
 724.7892456054688,
 724.4847412109375,
 732.81640625,
 733.6478271484375,
 732.2969360351562,
 724.7593383789062,
 737.9701538085938,
 724.9641723632812,
 724.968017578125,
 725.2062377929688]

In [230]:
pt_obs = gdf_observ[gdf_observ.SCM == '3314-231'].geometry#.coords[:]

In [231]:
[list(x.coords)[0] for x in pt_obs.geometry]

[(333344.32172328787, 7395282.674381973, 724.8905029296875),
 (333349.4469259395, 7395315.759854813, 724.7280883789062),
 (333353.60836743616, 7395345.880764694, 724.9146118164062),
 (333367.0835113299, 7395363.517350084, 724.698486328125),
 (333354.99551460164, 7395284.648125529, 724.8023071289062),
 (333357.17531729036, 7395273.947275966, 724.9680786132812),
 (333347.8616148932, 7395272.5601288015, 724.916748046875),
 (333361.336758787, 7395326.658868257, 724.8078002929688),
 (333399.3842238988, 7395358.365089184, 724.7515258789062),
 (333403.5456653954, 7395323.488246164, 724.8031005859375),
 (333409.68874569994, 7395362.52653068, 724.9634399414062),
 (333398.19524061406, 7395286.629764337, 724.8020629882812),
 (333388.6833743361, 7395285.837108814, 724.7892456054688),
 (333395.420946283, 7395324.082737806, 724.4847412109375),
 (333297.13166141073, 7395248.582299229, 732.81640625),
 (333278.7519614674, 7395273.204161419, 733.6478271484375),
 (333315.5113613541, 7395366.489808304, 73

In [232]:
# pt_obs

## Carregando dados LiDAR 3D

In [233]:
r = pdal.Pipeline(pipeline)
r.validate()
r.execute()
arrays = r.arrays

In [234]:
df = pd.DataFrame(arrays[0])

In [235]:
df_mds = \
df[(df.Classification == 2) | (df.Classification == 6)]\
                          [['X', 'Y', 'Z']]#\
# .astype({'X': 'int32', \
#         'Y': 'int32', \
#         'Z': 'int16'})

In [236]:
df_mdt = \
df[df.Classification == 2][['X', 'Y', 'Z']]

In [237]:
df

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime
0,333499.29,7395564.35,735.34,18,1,1,0,0,6,-14.0,17,6,356569.366065
1,333499.23,7395565.79,735.41,18,1,1,0,0,6,-14.0,17,6,356569.366090
2,333499.20,7395566.79,735.25,15,1,1,0,0,6,-14.0,17,6,356569.366105
3,333499.15,7395568.21,735.37,14,1,1,0,0,6,-14.0,17,6,356569.366129
4,333499.10,7395569.68,735.30,17,1,1,0,0,6,-15.0,17,6,356569.366153
...,...,...,...,...,...,...,...,...,...,...,...,...,...
471957,333523.36,7395570.78,735.52,2,1,2,1,0,19,13.0,40,6,363044.324524
471958,333521.03,7395570.91,731.07,5,1,1,1,0,6,13.0,40,6,363044.342732
471959,333520.96,7395571.00,732.50,8,1,1,1,0,6,13.0,40,6,363044.342740
471960,333521.09,7395571.84,731.83,18,1,1,1,0,6,13.0,40,6,363044.360909


In [238]:
# distance = np.sqrt((df_mdt.X - pt_obs[0])**2 + (df_mdt.Y - pt_obs[1])**2)

In [239]:
# distance.sort_values()[:3].index

In [240]:
# df_mdt

## Dividindo o domo celeste

In [241]:
fusos = 60
meridianos = 30

In [242]:
thetas, delta_theta = np.linspace(-np.pi, np.pi, fusos, retstep=True, endpoint=False)
thetas

array([-3.14159265, -3.0368729 , -2.93215314, -2.82743339, -2.72271363,
       -2.61799388, -2.51327412, -2.40855437, -2.30383461, -2.19911486,
       -2.0943951 , -1.98967535, -1.88495559, -1.78023584, -1.67551608,
       -1.57079633, -1.46607657, -1.36135682, -1.25663706, -1.15191731,
       -1.04719755, -0.9424778 , -0.83775804, -0.73303829, -0.62831853,
       -0.52359878, -0.41887902, -0.31415927, -0.20943951, -0.10471976,
        0.        ,  0.10471976,  0.20943951,  0.31415927,  0.41887902,
        0.52359878,  0.62831853,  0.73303829,  0.83775804,  0.9424778 ,
        1.04719755,  1.15191731,  1.25663706,  1.36135682,  1.46607657,
        1.57079633,  1.67551608,  1.78023584,  1.88495559,  1.98967535,
        2.0943951 ,  2.19911486,  2.30383461,  2.40855437,  2.51327412,
        2.61799388,  2.72271363,  2.82743339,  2.93215314,  3.0368729 ])

In [243]:
phis = np.linspace(1., 0., meridianos, retstep=True, endpoint=False)
phis

(array([1.        , 0.96666667, 0.93333333, 0.9       , 0.86666667,
        0.83333333, 0.8       , 0.76666667, 0.73333333, 0.7       ,
        0.66666667, 0.63333333, 0.6       , 0.56666667, 0.53333333,
        0.5       , 0.46666667, 0.43333333, 0.4       , 0.36666667,
        0.33333333, 0.3       , 0.26666667, 0.23333333, 0.2       ,
        0.16666667, 0.13333333, 0.1       , 0.06666667, 0.03333333]),
 -0.03333333333333333)

In [244]:
np.arccos(phis[0])

array([0.        , 0.25892154, 0.36720802, 0.45102681, 0.52231482,
       0.58568554, 0.64350111, 0.69716313, 0.74758435, 0.79539883,
       0.84106867, 0.88494336, 0.92729522, 0.96834169, 1.00826008,
       1.04719755, 1.0852782 , 1.12260819, 1.15927948, 1.19537272,
       1.23095942, 1.26610367, 1.30086353, 1.33529209, 1.36943841,
       1.40334825, 1.43706474, 1.47062891, 1.50408018, 1.53745682])

## Discretizando $\theta$ e $\phi$

In [245]:
## Dado um angulo theta t em radianos qual seu indice em 
## uma divisão de fusos a cada delta_theta

t = np.pi - 0.000000001
t = np.arctan2(0,-1)
np.floor((t + np.pi)/delta_theta).astype('int')

60

In [246]:
## Dado um angulo phi f em radianos qual seu indice em
## uma divisão de meridianos

f = np.pi/2
indice = np.floor(meridianos - (np.cos(f) / (1 / meridianos))).astype('int')
indice

29

## Calculando $\theta$ e $\phi$ para todos os pontos

Método por contagem de numero de grupos de $\theta$ e $\phi$

In [247]:
def svf_observ(x):
    coords = x.coords
    
    x_mds = df_mds.loc[df_mds.Z >= coords[0][2]].reset_index()
    
    x_mds['theta_idx'] = \
    np.floor(np.arctan2(\
              x_mds.X - coords[0][0],
              x_mds.Y - coords[0][1])/delta_theta).astype('int')
    
    x_mds['phi_idx'] = \
    np.floor(meridianos - (np.cos(\
    np.arctan2(\
               np.sqrt((x_mds.X - coords[0][0])**2 + \
                       (x_mds.Y - coords[0][1])**2), \
               x_mds.Z - coords[0][2])) / \
                           (1 / meridianos))).astype('int')

    
    svf = 1. - (x_mds.groupby(['theta_idx', 'phi_idx'])\
                .ngroups / (fusos * meridianos))
    
    return svf

In [248]:
pt_obs.apply(lambda x: svf_observ(x))

0     0.200000
3     0.256667
4     0.316667
5     0.316667
6     0.207222
7     0.215000
8     0.206111
9     0.253333
10    0.212222
11    0.167222
12    0.226667
13    0.203333
14    0.191111
15    0.161111
16    0.124444
17    0.109444
18    0.145556
19    0.315556
20    0.139444
21    0.438333
22    0.419444
23    0.507222
Name: geometry, dtype: float64

## Calculando $\theta$ e $\phi$ para todos os pontos

Método por máximo $\phi$ de grupos de $\theta$

In [249]:
def svf_observ(x):
    coords = x.coords
    
    x_mds = df_mds.loc[df_mds.Z >= coords[0][2]].reset_index()
    
    x_mds['theta_idx'] = \
    np.floor(np.arctan2(\
              x_mds.X - coords[0][0],
              x_mds.Y - coords[0][1])/delta_theta).astype('int')
    
    x_mds['phi'] = \
    np.arctan2(\
               np.sqrt((x_mds.X - coords[0][0])**2 + \
                       (x_mds.Y - coords[0][1])**2), \
               x_mds.Z - coords[0][2])

    
    svf = x_mds[['theta_idx', 'phi']].groupby(['theta_idx'])\
                .min()
    
    ## TODO
    # Calcular a área de cada um dos fusos theta_idx
    
    return svf.phi

In [250]:

pt_obs.apply(lambda x: svf_observ(x))

theta_idx,-30,-29,-28,-27,-26,-25,-24,-23,-22,-21,...,20,21,22,23,24,25,26,27,28,29
0,0.487757,0.494248,0.535641,0.586619,0.666392,0.738641,0.866750,1.060825,0.733360,0.472223,...,0.549333,0.539570,0.562925,0.665921,0.857174,1.314890,0.930102,0.652916,0.521783,0.496788
3,0.914798,0.960889,0.610734,0.429937,0.409477,0.415005,0.422732,0.439132,0.456723,0.473612,...,0.558552,0.487294,0.438434,0.443007,0.484484,0.534019,0.593979,0.882855,1.033030,0.920662
4,1.050662,1.078690,0.835702,0.833434,0.826308,0.702800,0.623218,0.587906,0.528433,0.459869,...,0.958436,0.974431,0.901811,0.991154,0.896316,0.856670,0.866701,0.867698,0.858158,0.982948
5,0.706059,1.047640,1.144905,1.005001,0.990461,0.986690,1.018238,1.080166,1.078122,0.987047,...,0.399221,0.335584,0.320037,0.385057,0.341477,0.406880,0.364705,0.453722,0.424278,0.438771
6,0.703728,0.664021,0.633282,0.641030,0.606340,0.618058,0.650763,0.820946,1.019862,1.000842,...,0.549837,0.511644,0.465484,0.443391,0.430135,0.438442,0.476820,1.437404,0.902603,0.769346
7,0.635867,0.589343,0.544084,0.478684,0.444967,0.471001,0.478537,0.468300,0.450678,0.445624,...,0.225463,0.223782,0.226223,0.242693,0.252926,0.392412,1.457910,1.419116,0.818841,0.714566
8,0.303445,0.248493,0.214589,0.239081,0.202678,0.215370,0.234723,0.231973,0.265867,0.381377,...,0.426724,0.504182,0.569906,0.699383,0.927261,1.306407,0.825748,0.530881,0.413613,0.336360
9,0.574840,0.973728,1.022329,1.078697,0.761594,0.664803,0.666466,0.682876,0.708068,0.731166,...,0.355790,0.335579,0.368454,0.387818,0.408810,0.474969,0.521080,0.570156,0.579061,0.551467
10,1.086027,0.330892,0.329548,0.264756,0.224708,0.269416,0.285107,0.239118,0.360090,0.293836,...,0.388878,0.399858,0.425151,0.447707,0.489339,0.545967,0.609246,0.692225,1.144185,1.125622
11,0.896945,0.870677,0.884047,0.831703,0.668236,0.651842,0.632156,0.640158,0.675204,0.815582,...,0.221390,0.229087,0.275391,0.305907,0.338917,0.335840,0.379382,0.473044,0.580579,0.690686


In [209]:
coords = pt_obs.iloc[1].coords

x_mds = df_mds.loc[df_mds.Z >= coords[0][2]].reset_index()

x_mds['theta_idx'] = \
np.floor(np.arctan2(\
          x_mds.X - coords[0][0],
          x_mds.Y - coords[0][1])/delta_theta).astype('int')

x_mds['phi'] = \
np.arctan2( \
           np.sqrt((x_mds.X - coords[0][0])**2 + \
                   (x_mds.Y - coords[0][1])**2), \
          x_mds.Z - coords[0][2])


svf = x_mds[['theta_idx', 'phi']].groupby(['theta_idx'])\
            .min()

In [215]:
# svf
np.arctan2(100, 100)

0.7853981633974483